##### Step 1: Initialize Workspace & Environment

In [52]:
from azureml.core import Workspace, Model
from azureml.core import Environment

# Initialize workspace
ws = Workspace.from_config()

# Get the trained model from registry
model_name= 'pima_model_SDKv1_02'
model = ws.models[model_name]
print(model.name, 'version', model.version)


# environment name
env_name= 'ENV-SDKv1'

# ---- create enviroment using .yaml file
# python_packages = Environment.from_conda_specification(env_name, '../dependencies/conda.yaml')
# # register the environment
# python_packages.register(workspace=ws)


pima_model_SDKv1_02 version 4


Step 3: Configure & Deploy Model as a Web Service

In [56]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

# Configure the scoring environment
service_env = Environment.get(workspace=ws, name=env_name)
service_env.inferencing_stack_version="latest" # to use latest version

inference_config = InferenceConfig(source_directory='../src/',
                                   entry_script= 'pima_scoreRealtimeEndpoint_SDKv1.py',
                                   environment=service_env
                                   )

# Configure the web service container
deployment_config = AciWebservice.deploy_configuration(cpu_cores=2, memory_gb=4)

# Deploy the model as a service
print('Deploying model...')
service_name = "pima-indians-sdkv1-service3"
service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, overwrite=True)
service.wait_for_deployment(True)
print(service.state)
print(service.get_logs())

Deploying model...
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-11-10 17:11:43+00:00 Creating Container Registry if not exists.
2023-11-10 17:11:44+00:00 Use the existing image.
2023-11-10 17:11:44+00:00 Generating deployment configuration.
2023-11-10 17:11:45+00:00 Submitting deployment to compute.
2023-11-10 17:11:52+00:00 Checking the status of deployment pima-indians-sdkv1-service3..
2023-11-10 17:13:26+00:00 Checking the status of inference endpoint pima-indians-sdkv1-service3.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
/bin/bash: /azureml-envs/azureml_dde9de8808e90dc357d096bbcd2dcc0e/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_dde9de8808e90dc357d096bbcd2dcc0e/lib/libtinfo.so.6: no version information available (required by /bin/bash)

/tmp/ipykernel_55961/4153428433.py:20: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, overwrite=True)


In [57]:
import json

x_new = [[0.0, 139.0, 62.0, 17.0, 210.0, 22.1, 0.207, 21.0]]

# print ('data: {}'.format(x_new[0]))

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# laoding Json file
# with open('../../data/sample_request.json', 'r') as f:
#   input_json = json.load(f)

# print(input_json)


# Call the web service, passing the input data (the web service will also accept the data in binary format)
predictions = service.run(input_data = input_json)
predictions


[0]